<a href="https://colab.research.google.com/github/Duchai263/Semantic-Analysis/blob/main/Semantic_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# installation

In [2]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 54.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 84.0 MB/s eta 0:00:00


In [3]:
pip install py_vncorenlp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 63.6 MB/s eta 0:00:00
  Created wheel for py_vncorenlp: filename=py_vncorenlp-0.1.4-py3-none-any.whl size=4323 sha256=636841e97db3a124b3b0e28df957fa9bff78a989ae08721d2a0c20188098a9a7
  Stored in directory: /root/.cache/pip/wheels/aa/f3/9d/5996e0385444a8f9825b24b94191ee94054e09b4e8f642d1c2
Successfully built py_vncorenlp


In [4]:
pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 44.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 71.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 kB 32.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 16.3 MB/s eta 0:00:00


In [5]:
import py_vncorenlp
try:
  py_vncorenlp.download_model(save_dir='./')
  rdrsegmenter = py_vncorenlp.VnCoreNLP(annotators=["wseg"], save_dir='./')
except:
  pass

# Data prepare

In [ ]:
data = []
with open("./train.txt","r") as f:
  for line in f:
    try:
      label,text = line.split('\t', 1)
      data.append({"label": label , "text": text})
    except:
      print(line)

negative    hơi chú trọng lý thuyết, cần TH nhiều hơn <3



In [ ]:
for idx,_ in enumerate(data):
  if 'neutral' in data[idx]['label']: data[idx]['label'] = 'neutral'

In [ ]:
for idx,_ in enumerate(data):
  data[idx]['text'] = ''.join(e for e in data[idx]['text'] if e.isalnum() or e == ' ')

In [ ]:
counter = {};
for each in data:
  label,text = each.values()
  if label in counter:
    counter[label] += 1
  else: counter[label] = 1

In [ ]:
counter

{'positive': 2361, 'negative': 1858, 'neutral': 780}

In [ ]:
txt = data[10]['text']
output = rdrsegmenter.word_segment(txt)

print(output[0])

kết_thúc môn rồi mà vẫn không biết mình đang học cái gì 3


In [ ]:
for idx,_ in enumerate(data):
  pp_text = rdrsegmenter.word_segment(data[idx]['text'])
  data[idx]['text'] = pp_text[0]

In [ ]:
data[0]['text']

'tài_liệu được bổ_sung nhiều và đầy_đủ trên moodle'

# Training

In [ ]:
id2label = {0: "negative", 1: "positive",2: "neutral"}
label2id = {"negative" : 0,"positive" :1,"neutral" :2}

In [ ]:
for idx,_ in enumerate(data):
  data[idx]['label'] = label2id[data[idx]['label']] 

In [ ]:
from sklearn.model_selection import train_test_split
import pandas as pd

dataframe = pd.DataFrame(data=data)
train_dataframe , val_dataframe = train_test_split(dataframe,train_size = 0.8,random_state=42)

In [ ]:
import datasets

train_data = datasets.Dataset.from_pandas(train_dataframe)
val_data = datasets.Dataset.from_pandas(val_dataframe)

In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer,AutoTokenizer

model = AutoModelForSequenceClassification.from_pretrained(
    "vinai/phobert-base", num_labels=3, id2label=id2label, label2id=label2id
)
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")

Some weights of the model checkpoint at vinai/phobert-base were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.decoder.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at vinai/phobert-base and are newly initialized: ['

In [ ]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True 
                     , padding="max_length"
                     )

In [ ]:
tokenized_train_data = train_data.map(preprocess_function, batched=True)
tokenized_val_data = val_data.map(preprocess_function, batched=True)

Map:   0%|          | 0/3999 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
import evaluate

accuracy = evaluate.load("accuracy")

import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [ ]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
training_args = TrainingArguments(
    output_dir="./my_model",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=8,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_data,
    eval_dataset=tokenized_val_data,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)
trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.700209,0.718000
2,No log,0.625256,0.756000
3,No log,0.621080,0.763000
4,0.710300,0.644078,0.749000
5,0.710300,0.631158,0.759000
6,0.710300,0.663436,0.749000
7,0.710300,0.669873,0.740000
8,0.500000,0.682221,0.734000


TrainOutput(global_step=1000, training_loss=0.6051372833251953, metrics={'train_runtime': 1454.0049, 'train_samples_per_second': 22.003, 'train_steps_per_second': 0.688, 'total_flos': 4208762229977088.0, 'train_loss': 0.6051372833251953, 'epoch': 8.0})

# Save & load model

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import torch
torch.save(model.state_dict(),"/content/phobert-8epochs_3.pth")

In [ ]:
!mv /content/phobert-8epochs_3.pth /content/drive/MyDrive/phobert-8epochs_3.pth

In [ ]:
import torch
path = "/content/drive/MyDrive/phobert-8epochs.pth"
model.load_state_dict(torch.load(path))

<All keys matched successfully>

# Inference

In [ ]:
texts = ["Thầy dạy không hay","Thầy dạy hay"]

In [ ]:
model.to('cuda')
def inference(text):
  model.eval()
  inputs = tokenizer(text,padding="max_length",truncation= True, return_tensors='pt').to('cuda')
  with torch.no_grad():
    logits = model(**inputs).logits
  predicted_class_id = logits.argmax().item()
  return model.config.id2label[predicted_class_id]

In [ ]:
print(inference(texts[0]))

negative


In [ ]:
inference_data = []

In [ ]:
with open("/content/data_phase_2.txt","r") as f:
  for line in f:
    try:
      text = line
      inference_data.append({"label": None , "text": text})
    except:
      print(line)

In [ ]:
def clean_data(data):
  for idx,_ in enumerate(data):
    data[idx]['text'] = ''.join(e for e in data[idx]['text'] if e.isalnum() or e == ' ')
    pp_text = rdrsegmenter.word_segment(data[idx]['text'])
    data[idx]['text'] = pp_text[0]
  return data

In [ ]:
inference_data = clean_data(inference_data)

In [ ]:
with open("/content/ans.txt","w") as f:
  for each in inference_data:
    f.write(inference(each['text'])+'\n')